In [1]:
# 동적 크롤링 라이브러리(selenium) 설치
! pip install selenium

  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 7.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 11.0 MB/s eta 0:00:0000:01
Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [2]:
# selnium 버전 업그레이드
! pip install selenium --upgrade

In [3]:
# 자동으로 웹 드라이버 불러오는 명령어 
!pip install webdriver-manager

In [2]:
# 동적 크롤링 라이브러리 세팅
from bs4 import BeautifulSoup
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys # import할 때는 k가 대문자 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [10]:
# 동적 크롤링 세팅 코드
options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

service = Service(ChromeDriverManager().install())

# 컴퓨터야 webdriver를 통해서 크롬브라우저 좀 실행 시켜줘 
# driver 변수를 통해 크롬창 제어 예정 
driver = wb.Chrome(service=service, options=options)
time.sleep(1)

url = "https://www.mentalhealth.go.kr/portal/disease/diseaseList.do"

# 원하는 페이지 가져오기 
driver.get(url)
time.sleep(1)

##### 첫 번째 파트
- 행렬 구조 활용한 데이터 수집

In [11]:
# 데이터를 담기 위한 빈 리스트 생성(질환명, 개요, 진단, 치료, 스스로 돕는 법)
nameList = [] 
summaryList = [] 
diagnosisList = []
healList = []
helpList = []

# 2행 6열 행렬 구조(위 부분 태그 구조)
rows = 2  
cols = 6 

# 정신질환 정보 수집 크롤링 코드(첫 번째 파트)
# 행렬의 각 요소에 대해 반복
for i in range(1, rows + 1):
    for j in range(1, cols + 1):
        # 동적으로 CSS 선택자 생성 및 클릭
        selector = f"#con_wrap > div.disease_part > ul:nth-child({i}) > li:nth-child({j})"
        driver.find_element(By.CSS_SELECTOR, selector).click()
        
        # 질환명 데이터 수집
        try:
            name = driver.find_element(By.CSS_SELECTOR, "div.tbl_tit").text
        except NoSuchElementException:
            name = "정보 없음"
        nameList.append(name)
        time.sleep(1)
    
        # 페이지 스크롤
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        body.send_keys(Keys.END)
        time.sleep(0.5)
        body.send_keys(Keys.HOME)
        time.sleep(0.5)
    
        ##################################
        # 1. 개요 - 정의, 원인, 역학 및 통계 수집
        define = cause = statistics = "정보 없음"
        
        try:
            define = driver.find_element(By.CSS_SELECTOR, "#tab1 > ul > li.trigger > div.accordi_con").text.strip() # 개요
        except:
            pass
            
        try:
            driver.find_element(By.CSS_SELECTOR, "#tab1 > ul > li:nth-child(2) > div.tit").click()
            cause = driver.find_element(By.CSS_SELECTOR, "#tab1 > ul > li:nth-child(2) > div.accordi_con").text.strip() # 원인
        except:
            pass
            
        try:
            driver.find_element(By.CSS_SELECTOR, "#tab1 > ul > li:nth-child(3) > div.tit").click()
            statistics = driver.find_element(By.CSS_SELECTOR, "#tab1 > ul > li:nth-child(3) > div.accordi_con").text.strip() # 역학 및 통계
        except:
            pass

        # 수집된 데이터를 리스트에 추가
        summaryList.append(f"정의: {define}\n원인: {cause}\n진단 및 통계: {statistics}")

        # 모두 스크롤한 후 가장 위로 스크롤
        body.send_keys(Keys.HOME)
        time.sleep(0.5)
    
        #################################    
        # 2. 진단 - 사례, 증상, 진단 기준, 검사
        example = symptom = diagcriteria = test = "정보 없음"

        # 진단 탭 클릭
        driver.find_element(By.CSS_SELECTOR, "#tab_area > div.tab_tit.step2 > h3").click()
        
        try:
            example = driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li.trigger > div.accordi_con").text.strip() # 사례
        except:
            pass
            
        try:
            driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li:nth-child(2) > div.tit").click()
            symptom = driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li:nth-child(2) > div.accordi_con").text.strip() # 증상
        except:
            pass
            
        try:
            driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li:nth-child(3) > div.tit").click()
            diagcriteria = driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li:nth-child(3) > div.accordi_con").text.strip() # 진단 기준
        except:
            pass
            
        try:
            driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li:nth-child(4) > div.tit").click()
            test = driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li:nth-child(4) > div.accordi_con").text.strip() # 검사
        except:
            pass

        # 수집된 데이터를 리스트에 추가
        diagnosisList.append(f"사례: {example}\n증상: {symptom}\n진단 기준: {diagcriteria}\n검사: {test}")

        # 모두 스크롤한 후 가장 위로 스크롤
        body.send_keys(Keys.HOME)
        time.sleep(0.5)
    
        ###################################    
        # 3. 치료 - 약물치료, 정신치료, 그외의 치료
        medication = mental = others = "정보 없음"

        # 치료 탭 클릭
        driver.find_element(By.CSS_SELECTOR, "#tab_area > div.tab_tit.step3 > h3").click()
        
        try:
            medication = driver.find_element(By.CSS_SELECTOR, "#tab3 > ul > li.trigger > div.accordi_con").text.strip() # 약물치료
        except:
            pass
            
        try:
            driver.find_element(By.CSS_SELECTOR, "#tab3 > ul > li:nth-child(2) > div.tit").click()
            mental = driver.find_element(By.CSS_SELECTOR, "#tab3 > ul > li:nth-child(2) > div.accordi_con").text.strip() # 정신치료
        except:
            pass
            
        try:
            driver.find_element(By.CSS_SELECTOR, "#tab3 > ul > li:nth-child(3) > div.tit").click()
            others = driver.find_element(By.CSS_SELECTOR, "#tab3 > ul > li:nth-child(3) > div.accordi_con").text.strip() # 그외의 치료
        except:
            pass

        # 수집된 데이터를 리스트에 추가
        healList.append(f"약물치료: {medication}\n정신치료: {mental}\n그외의 치료: {others}")

        # 모두 스크롤한 후 가장 위로 스크롤
        body.send_keys(Keys.HOME)
        time.sleep(0.5)
    
        #####################################
        # 4. 스스로 돕는 법 - 지원체계, 스스로 돕는 법
        support = myself = "정보 없음"

        # 스스로 돕는 법 탭 클릭
        driver.find_element(By.CSS_SELECTOR, "#tab_area > div.tab_tit.step4 > h3").click()
        
        try:
            support = driver.find_element(By.CSS_SELECTOR, "#tab4 > ul > li.trigger > div.accordi_con").text.strip() # 지원체계
        except:
            pass
            
        try:
            driver.find_element(By.CSS_SELECTOR, "#tab4 > ul > li:nth-child(2) > div.tit").click()
            myself = driver.find_element(By.CSS_SELECTOR, "#tab4 > ul > li:nth-child(2) > div.accordi_con").text.strip() # 스스로 돕는 법
        except:
            pass

        # 수집된 데이터를 리스트에 추가
        helpList.append(f"지원체계: {support}\n스스로 돕는 법: {myself}")
    
        time.sleep(0.5)

        
    
        # 데이터를 잘 가져왔는지 확인
        print(len(nameList), len(summaryList), len(diagnosisList), len(healList), len(helpList))
    
        # 데이터프레임 생성 및 CSV 파일로 저장
        df = pd.DataFrame({'질환명': nameList, '개요': summaryList, '진단': diagnosisList, '치료': healList, '스스로 돕는 법': helpList})
        df.to_csv("정신질환 정보 크롤링1.csv", encoding='utf-8-sig', index=False)
    
        # 이전 페이지로 돌아가기
        driver.back()
        time.sleep(2)

1 1 1 1 1
2 2 2 2 2
3 3 3 3 3
4 4 4 4 4
5 5 5 5 5
6 6 6 6 6
7 7 7 7 7
8 8 8 8 8
9 9 9 9 9
10 10 10 10 10
11 11 11 11 11
12 12 12 12 12


##### 두 번째 파트
- 순서를 활용한 데이터 수집

In [15]:
# 데이터를 담기 위한 빈 리스트 생성(질환명, 개요, 진단, 치료, 스스로 돕는 법)
nameList = [] 
summaryList = [] 
diagnosisList = []
healList = []
helpList = []

# 정신질환 정보 수집 크롤링 코드(두 번째 부분)
# 행렬의 각 요소에 대해 반복
for i in range(7, 45):
    # 동적으로 CSS 선택자 생성 및 클릭
    selector = f"#con_wrap > div.disease_part > ul.part_two > li:nth-child({i})"
    driver.find_element(By.CSS_SELECTOR, selector).click()
        
    # 질환명 데이터 수집
    name = driver.find_element(By.CSS_SELECTOR, "div.tbl_tit").text
    nameList.append(name)
    time.sleep(1)
    
    # 페이지 스크롤
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    # Body 태그에 End 키 이벤트를 전송하여 스크롤 다운
    body.send_keys(Keys.END)
    # 스크롤 후 페이지가 로드되는 것을 기다림
    time.sleep(1.5)
    
    # 모두 스크롤한 후 가장 위로 스크롤
    body.send_keys(Keys.HOME)
    time.sleep(0.5)
    
    ##################################
    # 1. 개요 - 정의, 원인, 역학 및 통계 수집
    define = cause = statistics = "정보 없음"
        
    try:
        define = driver.find_element(By.CSS_SELECTOR, "#tab1 > ul > li.trigger > div.accordi_con").text.strip() # 개요
    except:
        pass
            
    try:
        driver.find_element(By.CSS_SELECTOR, "#tab1 > ul > li:nth-child(2) > div.tit").click()
        cause = driver.find_element(By.CSS_SELECTOR, "#tab1 > ul > li:nth-child(2) > div.accordi_con").text.strip() # 원인
    except:
        pass
            
    try:
        driver.find_element(By.CSS_SELECTOR, "#tab1 > ul > li:nth-child(3) > div.tit").click()
        statistics = driver.find_element(By.CSS_SELECTOR, "#tab1 > ul > li:nth-child(3) > div.accordi_con").text.strip() # 역학 및 통계
    except:
        pass

    # 수집된 데이터를 리스트에 추가
    summaryList.append(f"정의: {define}\n원인: {cause}\n진단 및 통계: {statistics}")

    # 모두 스크롤한 후 가장 위로 스크롤
    body.send_keys(Keys.HOME)
    time.sleep(0.5)
    
    #################################    
    # 2. 진단 - 사례, 증상, 진단 기준, 검사
    example = symptom = diagcriteria = test = "정보 없음"

    # 진단 탭 클릭
    driver.find_element(By.CSS_SELECTOR, "#tab_area > div.tab_tit.step2 > h3").click()
        
    try:
        example = driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li.trigger > div.accordi_con").text.strip() # 사례
    except:
        pass
            
    try:
        driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li:nth-child(2) > div.tit").click()
        symptom = driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li:nth-child(2) > div.accordi_con").text.strip() # 증상
    except:
        pass
            
    try:
        driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li:nth-child(3) > div.tit").click()
        diagcriteria = driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li:nth-child(3) > div.accordi_con").text.strip() # 진단 기준
    except:
        pass
            
    try:
        driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li:nth-child(4) > div.tit").click()
        test = driver.find_element(By.CSS_SELECTOR, "#tab2 > ul > li:nth-child(4) > div.accordi_con").text.strip() # 검사
    except:
        pass

    # 수집된 데이터를 리스트에 추가
    diagnosisList.append(f"사례: {example}\n증상: {symptom}\n진단 기준: {diagcriteria}\n검사: {test}")

    # 모두 스크롤한 후 가장 위로 스크롤
    body.send_keys(Keys.HOME)
    time.sleep(0.5)
    
    ###################################    
    # 3. 치료 - 약물치료, 정신치료, 그외의 치료
    medication = mental = others = "정보 없음"

    # 치료 탭 클릭
    driver.find_element(By.CSS_SELECTOR, "#tab_area > div.tab_tit.step3 > h3").click()
        
    try:
        medication = driver.find_element(By.CSS_SELECTOR, "#tab3 > ul > li.trigger > div.accordi_con").text.strip() # 약물치료
    except:
        pass
            
    try:
        driver.find_element(By.CSS_SELECTOR, "#tab3 > ul > li:nth-child(2) > div.tit").click()
        mental = driver.find_element(By.CSS_SELECTOR, "#tab3 > ul > li:nth-child(2) > div.accordi_con").text.strip() # 정신치료
    except:
        pass
            
    try:
        driver.find_element(By.CSS_SELECTOR, "#tab3 > ul > li:nth-child(3) > div.tit").click()
        others = driver.find_element(By.CSS_SELECTOR, "#tab3 > ul > li:nth-child(3) > div.accordi_con").text.strip() # 그외의 치료
    except:
        pass

    # 수집된 데이터를 리스트에 추가
    healList.append(f"약물치료: {medication}\n정신치료: {mental}\n그외의 치료: {others}")

    # 모두 스크롤한 후 가장 위로 스크롤
    body.send_keys(Keys.HOME)
    time.sleep(0.5)
    
    #####################################
    # 4. 스스로 돕는 법 - 지원체계, 스스로 돕는 법
    support = myself = "정보 없음"

    # 스스로 돕는 법 탭 클릭
    driver.find_element(By.CSS_SELECTOR, "#tab_area > div.tab_tit.step4 > h3").click()
        
    try:
        support = driver.find_element(By.CSS_SELECTOR, "#tab4 > ul > li.trigger > div.accordi_con").text.strip() # 지원체계
    except:
        pass
            
    try:
        driver.find_element(By.CSS_SELECTOR, "#tab4 > ul > li:nth-child(2) > div.tit").click()
        myself = driver.find_element(By.CSS_SELECTOR, "#tab4 > ul > li:nth-child(2) > div.accordi_con").text.strip() # 스스로 돕는 법
    except:
        pass

    # 수집된 데이터를 리스트에 추가
    helpList.append(f"지원체계: {support}\n스스로 돕는 법: {myself}")
    
    time.sleep(0.5)
    
    
        
    
    # 데이터를 잘 가져왔는지 확인
    print(len(nameList), len(summaryList), len(diagnosisList), len(healList), len(helpList))
    
    # 데이터프레임 생성 및 CSV 파일로 저장
    df = pd.DataFrame({'질환명': nameList, '개요': summaryList, '진단': diagnosisList, '치료': healList, '스스로 돕는 법': helpList})
    df.to_csv("정신질환 정보 크롤링2.csv", encoding = 'utf-8-sig', index=False)
    
    # 이전 페이지로 돌아가기
    driver.back()
    time.sleep(2)

1 1 1 1 1
2 2 2 2 2
3 3 3 3 3
4 4 4 4 4
5 5 5 5 5
6 6 6 6 6
7 7 7 7 7
8 8 8 8 8
9 9 9 9 9
10 10 10 10 10
11 11 11 11 11
12 12 12 12 12
13 13 13 13 13
14 14 14 14 14
15 15 15 15 15
16 16 16 16 16
17 17 17 17 17
18 18 18 18 18
19 19 19 19 19
20 20 20 20 20
21 21 21 21 21
22 22 22 22 22
23 23 23 23 23
24 24 24 24 24
25 25 25 25 25
26 26 26 26 26
27 27 27 27 27
28 28 28 28 28
29 29 29 29 29
30 30 30 30 30
31 31 31 31 31
32 32 32 32 32
33 33 33 33 33
34 34 34 34 34
35 35 35 35 35
36 36 36 36 36
37 37 37 37 37
38 38 38 38 38
